In [2]:

import matplotlib.pyplot as plt
import skimage
import scipy
from skimage import transform
import PIL
from PIL import Image
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import cv2

# 1

In [3]:
# Read the RGB images
mountain1_img = cv2.imread('Task2/images/mountain1.jpg')
mountain2_img = cv2.imread('Task2/images/mountain2.jpg')

# 2

In [4]:

def homography(u1, v1, u2, v2):
    # Construct the matrix A
    A = []
    for i in range(len(u1)):
        A.append([-u1[i], -v1[i], -1, 0, 0, 0, u1[i]*u2[i], v1[i]*u2[i], u2[i]])
        A.append([0, 0, 0, -u1[i], -v1[i], -1, u1[i]*v2[i], v1[i]*v2[i], v2[i]])

    A = np.asarray(A)

    # Singular Value Decomposition (SVD) of A
    _, _, V = np.linalg.svd(A)

    # The homography matrix is the last column of V
    H = V[-1].reshape(3, 3)

    return H


# 3

In [5]:


def compute_normalisation_matrix(points):
    """
    Compute normalisation matrix for homogeneous point coordinates.

    Arguments:
    points: numpy array of shape (n, 3) containing homogeneous point coordinates

    Returns:
    T: normalisation matrix
    """

    # Calculate mean of x and y coordinates
    mean_x = np.mean(points[:, 0])
    mean_y = np.mean(points[:, 1])

    # Calculate scale factor
    dist_mean = np.sqrt((points[:, 0] - mean_x) ** 2 + (points[:, 1] - mean_y) ** 2)
    mean_dist = np.mean(dist_mean)
    scale = np.sqrt(2) / mean_dist

    # Create normalisation matrix
    T = np.array([[scale, 0, -scale * mean_x],
                  [0, scale, -scale * mean_y],
                  [0, 0, 1]])

    return T


In [6]:
def homography_w_normalisation(u1, v1, u2, v2):
    """
    Compute homography matrix with normalisation.

    Arguments:
    u1, v1: normalised (u,v) coordinates from image 1
    u2, v2: normalised (u,v) coordinates from image 2

    Returns:
    H: the 3×3 homography matrix that warps unnormalised coordinates from image 1 into
    unnormalised coordinates from image 2
    """

    # Convert coordinates to homogeneous form
    points1 = np.column_stack((u1, v1, np.ones_like(u1)))
    points2 = np.column_stack((u2, v2, np.ones_like(u2)))

    # Compute normalisation matrices
    T1 = compute_normalisation_matrix(points1)
    T2 = compute_normalisation_matrix(points2)

    # Apply normalisation to points
    norm_points1 = np.dot(points1, T1.T)
    norm_points2 = np.dot(points2, T2.T)

    # Apply DLT algorithm to estimate homography matrix
    H_norm = homography(norm_points1[:, 0], norm_points1[:, 1], norm_points2[:, 0], norm_points2[:, 1])

    # Denormalise H
    H = np.dot(np.linalg.inv(T2), np.dot(H_norm, T1))

    return H


In [7]:
import numpy as np

def homography(u1, v1, u2, v2):
    """
    Dummy function for estimating homography matrix.
    This function needs to be replaced with the actual implementation.
    """
    # Dummy homography matrix for testing
    return np.array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]])

# Define the compute_normalisation_matrix function
def compute_normalisation_matrix(points):
    """
    Compute normalisation matrix for homogeneous point coordinates.

    Arguments:
    points: numpy array of shape (n, 3) containing homogeneous point coordinates

    Returns:
    T: normalisation matrix
    """

    # Calculate mean of x and y coordinates
    mean_x = np.mean(points[:, 0])
    mean_y = np.mean(points[:, 1])

    # Calculate scale factor
    dist_mean = np.sqrt((points[:, 0] - mean_x) ** 2 + (points[:, 1] - mean_y) ** 2)
    mean_dist = np.mean(dist_mean)
    scale = np.sqrt(2) / mean_dist

    # Create normalisation matrix
    T = np.array([[scale, 0, -scale * mean_x],
                  [0, scale, -scale * mean_y],
                  [0, 0, 1]])

    return T

# Define the homography_w_normalisation function
def homography_w_normalisation(u1, v1, u2, v2):
    """
    Compute homography matrix with normalisation.

    Arguments:
    u1, v1: normalised (u,v) coordinates from image 1
    u2, v2: normalised (u,v) coordinates from image 2

    Returns:
    H: the 3×3 homography matrix that warps unnormalised coordinates from image 1 into
    unnormalised coordinates from image 2
    """

    # Convert coordinates to homogeneous form
    points1 = np.column_stack((u1, v1, np.ones_like(u1)))
    points2 = np.column_stack((u2, v2, np.ones_like(u2)))

    # Compute normalisation matrices
    T1 = compute_normalisation_matrix(points1)
    T2 = compute_normalisation_matrix(points2)

    # Apply normalisation to points
    norm_points1 = np.dot(points1, T1.T)
    norm_points2 = np.dot(points2, T2.T)

    # Apply DLT algorithm to estimate homography matrix
    H_norm = homography(norm_points1[:, 0], norm_points1[:, 1], norm_points2[:, 0], norm_points2[:, 1])

    # Denormalise H
    H = np.dot(np.linalg.inv(T2), np.dot(H_norm, T1))

    return H

# Test the homography_w_normalisation function with sample coordinates
u1 = np.array([0.1, 0.2, 0.3])
v1 = np.array([0.3, 0.5, 0.8])
u2 = np.array([0.15, 0.25, 0.35])
v2 = np.array([0.35, 0.55, 0.85])

# Calculate homography matrix with normalisation
H = homography_w_normalisation(u1, v1, u2, v2)
print("Homography matrix with normalisation:")
print(H)


Homography matrix with normalisation:
[[1.   0.   0.05]
 [0.   1.   0.05]
 [0.   0.   1.  ]]


# 4

In [7]:

def find_matching_keypoints(image1, image2):
    """
    Find matching keypoints between two images using SIFT.

    Arguments:
    image1: numpy array representing the first image
    image2: numpy array representing the second image

    Returns:
    keypoints1, keypoints2: lists of keypoints found in image1 and image2, respectively
    matches: list of matching keypoint pairs
    """

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

    # Initialize matcher
    bf = cv2.BFMatcher()

    # Match descriptors
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Get corresponding keypoints
    matched_keypoints1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
    matched_keypoints2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

    return matched_keypoints1, matched_keypoints2

def sample_corresponding_points(keypoints1, keypoints2, seed=42):
    """
    Sample half of the corresponding points and store as .npy files.

    Arguments:
    keypoints1: numpy array of keypoints from image 1
    keypoints2: numpy array of keypoints from image 2
    seed: random seed for reproducibility

    Returns:
    sampled_keypoints1, sampled_keypoints2: sampled corresponding keypoints
    """

    # Set random seed for reproducibility
    np.random.seed(seed)

    # Sample half of the corresponding points
    num_points = min(len(keypoints1), len(keypoints2))
    indices = np.random.choice(num_points, size=num_points // 2, replace=False)

    # Select sampled keypoints
    sampled_keypoints1 = keypoints1[indices]
    sampled_keypoints2 = keypoints2[indices]

    # Save keypoints as .npy files
    np.save('sampled_keypoints1.npy', sampled_keypoints1)
    np.save('sampled_keypoints2.npy', sampled_keypoints2)

    return sampled_keypoints1, sampled_keypoints2

# Load images
image1 = cv2.imread('Task2/images/mountain1.jpg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('Task2/images/mountain2.jpg', cv2.IMREAD_GRAYSCALE)


# Find matching keypoints
keypoints1, keypoints2 = find_matching_keypoints(image1, image2)

# Sample corresponding points and save as .npy files
sampled_keypoints1, sampled_keypoints2 = sample_corresponding_points(keypoints1, keypoints2)


# 5

In [8]:


def compute_normalisation_matrix(points):
    """
    Compute normalisation matrix for homogeneous point coordinates.

    Arguments:
    points: numpy array of shape (n, 3) containing homogeneous point coordinates

    Returns:
    T: normalisation matrix
    """

    # Calculate mean of x and y coordinates
    mean_x = np.mean(points[:, 0])
    mean_y = np.mean(points[:, 1])

    # Calculate scale factor
    dist_mean = np.sqrt((points[:, 0] - mean_x) ** 2 + (points[:, 1] - mean_y) ** 2)
    mean_dist = np.mean(dist_mean)
    scale = np.sqrt(2) / mean_dist

    # Create normalisation matrix
    T = np.array([[scale, 0, -scale * mean_x],
                  [0, scale, -scale * mean_y],
                  [0, 0, 1]])

    return T

def homography_w_normalisation_ransac(u1, v1, u2, v2, threshold=4, max_iterations=1000):
    """
    Estimate homography matrix with normalisation using RANSAC.

    Arguments:
    u1, v1: normalised (u,v) coordinates from image 1
    u2, v2: normalised (u,v) coordinates from image 2
    threshold: RANSAC inlier threshold
    max_iterations: maximum number of RANSAC iterations

    Returns:
    H: the 3×3 homography matrix estimated by RANSAC
    """

    best_H = None
    max_inliers = 0

    # Convert coordinates to homogeneous form
    points1 = np.column_stack((u1, v1, np.ones_like(u1)))
    points2 = np.column_stack((u2, v2, np.ones_like(u2)))

    for _ in range(max_iterations):
        # Randomly select 4 correspondences
        indices = np.random.choice(len(u1), size=4, replace=False)
        src_pts = points1[indices, :]
        dst_pts = points2[indices, :]

        # Compute homography using selected correspondences
        H = cv2.findHomography(src_pts[:, :2], dst_pts[:, :2])[0]

        # Calculate reprojection error
        projected_pts = np.dot(points1, H.T)
        projected_pts /= projected_pts[:, 2:]
        errors = np.sqrt(np.sum((points2[:, :2] - projected_pts[:, :2]) ** 2, axis=1))

        # Count inliers
        inliers = np.sum(errors < threshold)

        # Update best homography if current model is better
        if inliers > max_inliers:
            max_inliers = inliers
            best_H = H

    return best_H

# Load sampled keypoints from previous question
sampled_keypoints1 = np.load('sampled_keypoints1.npy')
sampled_keypoints2 = np.load('sampled_keypoints2.npy')

# Perform RANSAC to estimate homography
H_ransac = homography_w_normalisation_ransac(sampled_keypoints1[:, 0], sampled_keypoints1[:, 1],
                                             sampled_keypoints2[:, 0], sampled_keypoints2[:, 1])

print("Homography matrix estimated by RANSAC:")
print(H_ransac)


Homography matrix estimated by RANSAC:
[[ 5.80530290e-01  1.37622115e-01  2.58412010e+02]
 [-2.87159724e-01  8.64004025e-01  5.15256161e+01]
 [-7.45946837e-04 -1.02565007e-04  1.00000000e+00]]


# 6

In [10]:


def warp_image(img, H, output_size):
    """
    Warp an image according to a given homography matrix.

    Arguments:
    img: input image to be warped
    H: homography matrix
    output_size: size of the output image (width, height)

    Returns:
    warped_img: warped image
    """

    # Warp image using given homography
    warped_img = cv2.warpPerspective(img, H, output_size)

    return warped_img

# Load images
mountain1 = cv2.imread('Task2/images/mountain1.jpg')
mountain2 = cv2.imread('Task2/images/mountain2.jpg')

# Compute dimensions for the output image
output_width = mountain1.shape[1] + mountain2.shape[1]
output_height = max(mountain1.shape[0], mountain2.shape[0])

# Compute homography between mountain1 and mountain2
H = homography_w_normalisation_ransac(sampled_keypoints1[:, 0], sampled_keypoints1[:, 1],
                                       sampled_keypoints2[:, 0], sampled_keypoints2[:, 1])

# Warp mountain1 image according to the calculated homography
warped_mountain1 = warp_image(mountain1, H, (output_width, output_height))

# Stitch warped mountain1 image with mountain2 image
stitched_image = np.zeros((output_height, output_width, 3), dtype=np.uint8)
stitched_image[:mountain2.shape[0], :mountain2.shape[1]] = mountain2
stitched_image[:, mountain2.shape[1]:] = warped_mountain1

# Display the stitched image
cv2.imshow('Stitched Image', stitched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: could not broadcast input array from shape (374,1034,3) into shape (374,517,3)

In [13]:
import numpy as np
import cv2

def warp_image(img, H, output_size):
    """
    Warp an image according to a given homography matrix.

    Arguments:
    img: input image to be warped
    H: homography matrix
    output_size: size of the output image (width, height)

    Returns:
    warped_img: warped image
    """

    # Create an empty canvas for the warped image
    warped_img = np.zeros(output_size, dtype=img.dtype)

    # Invert homography matrix for forward warping
    H_inv = np.linalg.inv(H)

    # Iterate over each pixel in the output image
    for y in range(output_size[1]):
        for x in range(output_size[0]):
            # Map output pixel back to input pixel using inverse homography
            p = np.dot(H_inv, np.array([x, y, 1]))
            p = p / p[2]
            src_x, src_y = int(p[0]), int(p[1])

            # Check if mapped pixel is within the bounds of the input image
            if 0 <= src_x < img.shape[1] and 0 <= src_y < img.shape[0]:
                # Copy pixel value from input image to output image
                warped_img[y, x] = img[src_y, src_x]

    return warped_img

# Load images
mountain1 = cv2.imread('Task2/images/mountain1.jpg')
mountain2 = cv2.imread('Task2/images/mountain2.jpg')

# Compute dimensions for the output image
output_width = mountain1.shape[1] + mountain2.shape[1]
output_height = max(mountain1.shape[0], mountain2.shape[0])

# Compute homography between mountain1 and mountain2
H = homography_w_normalisation_ransac(sampled_keypoints1[:, 0], sampled_keypoints1[:, 1],
                                       sampled_keypoints2[:, 0], sampled_keypoints2[:, 1])

# Warp mountain1 image according to the calculated homography
warped_mountain1 = warp_image(mountain1, H, (output_width, output_height))

# Create an empty canvas for stitching
stitched_image = np.zeros((output_height, output_width, 3), dtype=np.uint8)

# Copy mountain2 image to the left half of the canvas
stitched_image[:mountain2.shape[0], :mountain2.shape[1]] = mountain2

# Blend warped mountain1 image with the right half of the canvas
stitched_image[:, mountain2.shape[1]:] = warped_mountain1[:, :mountain2.shape[1]]

# Display the stitched image
cv2.imshow('Stitched Image', stitched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


IndexError: index 414 is out of bounds for axis 1 with size 374

In [12]:


def warp_image(img, H, output_size):
    """
    Warp an image according to a given homography matrix.

    Arguments:
    img: input image to be warped
    H: homography matrix
    output_size: size of the output image (width, height)

    Returns:
    warped_img: warped image
    """

    # Warp image using given homography
    warped_img = cv2.warpPerspective(img, H, output_size)

    return warped_img

# Load images
mountain1 = cv2.imread('Task2/images/mountain1.jpg')
mountain2 = cv2.imread('Task2/images/mountain2.jpg')

# Compute dimensions for the output image
output_width = mountain1.shape[1] + mountain2.shape[1]
output_height = max(mountain1.shape[0], mountain2.shape[0])

# Compute homography between mountain1 and mountain2
H = homography_w_normalisation_ransac(sampled_keypoints1[:, 0], sampled_keypoints1[:, 1],
                                       sampled_keypoints2[:, 0], sampled_keypoints2[:, 1])

# Warp mountain1 image according to the calculated homography
warped_mountain1 = warp_image(mountain1, H, (output_width, output_height))

# Create an empty canvas for stitching
stitched_image = np.zeros((output_height, output_width, 3), dtype=np.uint8)

# Copy mountain2 image to the left half of the canvas
stitched_image[:mountain2.shape[0], :mountain2.shape[1]] = mountain2

# Blend warped mountain1 image with the right half of the canvas
stitched_image[:, mountain2.shape[1]:] = warped_mountain1[:, :mountain2.shape[1]]

# Display the stitched image
cv2.imshow('Stitched Image', stitched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\XUDU\AppData\Local\Temp\ipykernel_29640\2555800168.py:60: RuntimeWarning: divide by zero encountered in divide
  projected_pts /= projected_pts[:, 2:]
C:\Users\XUDU\AppData\Local\Temp\ipykernel_29640\2555800168.py:60: RuntimeWarning: invalid value encountered in divide
  projected_pts /= projected_pts[:, 2:]


# 7

In [15]:

def select_correspondences(image1, image2):
    """
    Manually select matching points from two images.

    Arguments:
    image1: first image
    image2: second image

    Returns:
    correspondences: list of corresponding points [(x1, y1, x2, y2), ...]
    """
    cv2.imshow('Image 1', image1)
    cv2.imshow('Image 2', image2)
    correspondences = []
    while True:
        print("Click corresponding points on both images. Press 'q' to finish.")
        point1 = cv2.ginput(1, timeout=-1)
        if not point1:
            break
        point2 = cv2.ginput(1, timeout=-1)
        correspondences.append((point1[0][0], point1[0][1], point2[0][0], point2[0][1]))
        cv2.circle(image1, (int(point1[0][0]), int(point1[0][1])), 5, (0, 255, 0), -1)
        cv2.circle(image2, (int(point2[0][0]), int(point2[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow('Image 1', image1)
        cv2.imshow('Image 2', image2)
    cv2.destroyAllWindows()
    return correspondences

# Load images
image1 = cv2.imread('Task2/images/mountain1.jpg')
image2 = cv2.imread('Task2/images/mountain2.jpg')

# Manually select correspondences
correspondences = select_correspondences(image1.copy(), image2.copy())

# Extract corresponding points
points1 = np.array([(c[0], c[1], 1) for c in correspondences])
points2 = np.array([(c[2], c[3], 1) for c in correspondences])

# Calculate homography matrix
H = homography_w_normalisation(points1[:, 0], points1[:, 1], points2[:, 0], points2[:, 1])

# Use H for rectification or further analysis


Click corresponding points on both images. Press 'q' to finish.


AttributeError: module 'cv2' has no attribute 'ginput'

: 

The rectified results, achieved through methods like homography transformation, can be influenced by various factors both theoretically and empirically. Let's discuss them:

Theoretical Factors:
Accuracy of Correspondences: The accuracy of correspondences between images directly affects the quality of rectified results. If the correspondences are inaccurate or contain outliers, the estimated homography matrix will be incorrect, leading to distorted rectified images.
Homography Estimation Method: The method used to estimate the homography matrix also plays a significant role. Different algorithms, like Direct Linear Transformation (DLT) or RANSAC, have different strengths and weaknesses in handling noise, outliers, and geometric transformations, which can affect the rectified results.
Choice of Normalisation: The normalisation step in homography estimation helps in stabilising the computation and improving the accuracy of the estimated matrix. However, the choice of normalisation method can affect the final rectified results.
Empirical Factors:
Quality of Feature Detection and Matching: The accuracy and robustness of feature detection and matching algorithms, such as SIFT or SURF, impact the quality of correspondences. Higher quality correspondences lead to better rectified results.
Homography Model Assumptions: The homography model assumes a planar scene and a pinhole camera model. Deviations from these assumptions, such as non-planar scenes or lens distortion, can affect the accuracy of rectified results.
Image Distortions: Distortions present in the input images, such as lens distortion or perspective distortions, can impact the accuracy of homography estimation and subsequently affect the rectified results.
Noise and Occlusions: Noise in the images and occlusions in the scene can introduce errors in feature detection and matching, leading to inaccuracies in homography estimation and rectified results.
To empirically assess the rectified results, one can manually select ground-truth correspondences between images and use functions like homography_w_normalisation to calculate the homography matrix. Comparing the rectified images obtained using the estimated homography with the ground truth can provide insights into the accuracy and effectiveness of the rectification process. Additionally, quantitative metrics such as reprojection error or visual inspection can be used to evaluate the rectified results.